# An Example of Calling Batch Transform Job on the model SageMaker estimator made

In [1]:
%store -r

In [6]:
import boto3

from sagemaker.transformer import Transformer
import pandas as pd
import sagemaker

from time import strftime, gmtime

sess = sagemaker.Session()

role = sagemaker.get_execution_role()
client = boto3.client('sagemaker')

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
transform_job_name = 'churn-batch-transform-' + timestamp_prefix

output_prefix = 'output'
output_path = 's3://{}/{}/{}'.format(bucket, prefix, output_prefix)

# Make Transformer object
transformer = Transformer(base_transform_job_name = transform_job_name,
                          model_name = model_name,
                          instance_count=1,
                          instance_type='ml.m4.xlarge',
                          output_path= output_path
                         )



In [7]:
import os

s3_input_test_url = s3_input_test.config['DataSource']['S3DataSource']['S3Uri']
s3_input_test_url = os.path.join(s3_input_test_url, 'batch_transform_test.csv')
print(s3_input_test_url)

s3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/rawtest/batch_transform_test.csv


In [8]:
! aws s3 ls {s3_input_test_url} --recursive

2020-07-15 03:34:44      29043 sagemaker/customer-churn/rawtest/batch_transform_test.csv


In [9]:

transformer.transform(s3_input_test_url,
                      content_type='text/csv'
                     )



In [10]:
transformer.wait()

.........................Processing /opt/ml/code
  Created wheel for preprocessing: filename=preprocessing-1.0.0-py2.py3-none-any.whl size=9703 sha256=7a2c5cf5d4d5e2916e496bb3891be7a23e47443f58aef274d2088ba57b94d15b
  Stored in directory: /tmp/pip-ephem-wheel-cache-vza_gqto/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
Successfully built preprocessing
Processing /opt/ml/code
  Created wheel for preprocessing: filename=preprocessing-1.0.0-py2.py3-none-any.whl size=9703 sha256=7a2c5cf5d4d5e2916e496bb3891be7a23e47443f58aef274d2088ba57b94d15b
  Stored in directory: /tmp/pip-ephem-wheel-cache-vza_gqto/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
Successfully built preprocessing
Arguments: serve
[2020-07-15 07:47:34 +0000] [1] [INFO] Starting gunicorn 19.7.1
[2020-07-15 07:47:34 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2020-07-15 07:47:34 +0000] [1] [INFO] Using worker: gevent
[2020-07-15 07:47:34 +0000] [36] [INFO] Booting worker wi

In [16]:
! aws s3 ls {output_path} --recursive

2020-07-15 07:10:13       1961 sagemaker/customer-churn/output-transform-test/batch_transform_test.csv.out
2020-07-15 07:47:47       1961 sagemaker/customer-churn/output/batch_transform_test.csv.out
2020-07-15 06:40:43      34149 sagemaker/customer-churn/output/xgboost-2020-07-15-06-37-54-125/output/model.tar.gz


In [20]:
import os
os.makedirs('output', exist_ok=True)

! aws s3 cp {output_path}/batch_transform_test.csv.out 'output/batch_transform_test_output.csv'

download: s3://sagemaker-us-east-2-057716757052/sagemaker/customer-churn/output/batch_transform_test.csv.out to output/batch_transform_test_output.csv


In [21]:

df = pd.read_csv("output/batch_transform_test_output.csv", names=["Churn"])

# print(df.info)
# df

df.Churn.value_counts()

False    291
True      43
Name: Churn, dtype: int64